In [2]:
import pandas as pd

In [3]:
data_yf = pd.read_csv('data/all_stock_data.csv')
data_yf.isna().any().any()
data_yf

,Date,Open,High,Low,Close,Adj Close,Volume,Ticker,macd,macd_signal,...,kc_lag_4,kc_lag_12,vwap_lag_4,vwap_lag_12,eom_lag_4,eom_lag_12,vpt_lag_4,vpt_lag_12,gain_loss_pct,win
0,2005-01-01,1.156786,1.390893,1.117857,1.373214,1.162562,38.626341,AAPL,0.019306,0.603729,...,1.168618,1.153541,1.211573,1.167815,0.024431,-0.227915,-5.001520,-5.005480,0.187094,1
1,2005-02-01,1.375893,1.622857,1.367500,1.602143,1.356373,32.189698,AAPL,-0.697760,0.332323,...,1.179566,1.152156,1.235185,1.135996,0.075228,0.496524,-4.999812,-5.005301,0.164439,1
2,2005-03-01,1.606786,1.611071,1.386786,1.488214,1.259921,21.954315,AAPL,-1.251914,-0.004435,...,1.200310,1.150016,1.287250,1.112824,0.581598,0.159055,-4.996778,-5.000109,-0.073795,-1
3,2005-04-01,1.503214,1.587500,1.214286,1.287857,1.090299,29.056338,AAPL,-1.672339,-0.364604,...,1.225616,1.150026,1.348575,1.110430,0.072348,0.406995,-4.996955,-4.998494,-0.143264,-1
4,2005-05-01,1.293214,1.462143,1.182500,1.420000,1.202171,19.207524,AAPL,-1.982337,-0.719664,...,1.145532,1.159878,-0.080819,1.117653,-10.441043,0.410987,-6.138958,-4.996653,0.098039,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
90511,2023-08-01,403.059998,419.000000,375.920013,400.200012,400.200012,-0.218175,BIO,-1.313996,-1.032681,...,2.767504,3.250799,2.414665,3.306907,-0.407752,0.128764,1.392980,1.392371,-0.007096,-1
90512,2023-09-01,403.940002,406.440002,349.269989,358.450012,358.450012,-0.218753,BIO,-1.420252,-1.132780,...,2.656697,3.209133,2.320586,3.203550,-1.924982,-2.363448,1.392934,1.392350,-0.112616,-1
90513,2023-10-01,355.540009,366.299988,261.589996,275.279999,275.279999,-0.215411,BIO,-1.703035,-1.273907,...,2.539979,3.140147,2.236475,3.003707,-0.491761,-2.682393,1.392938,1.392308,-0.225741,-1
90514,2023-11-01,274.170013,314.820007,267.859985,304.920013,304.920013,-0.213842,BIO,-1.828312,-1.413855,...,2.434953,3.089418,2.173890,2.825007,0.644527,-0.077603,1.392951,1.392354,0.112157,1


In [28]:
#changing granularity to monthly
import requests
import pandas as pd
import time
from datetime import datetime

# take the top 300 stock tickers from our complete dataframe 
tickers = data_yf['Ticker'].unique()

# Set the time_from parameter to January 1, 2010, 12:00 AM
time_from = '20100101T0000'

# Get the current time in YYYYMMDDTHHMM format
time_to = '20220301T0000'

# Function to fetch news sentiment data for a ticker within a time range
def fetch_news_sentiment(ticker, start_time, end_time):
    url = f'https://www.alphavantage.co/query?function=NEWS_SENTIMENT&tickers={ticker}&apikey={apikey}&time_from={start_time}&time_to={end_time}&sort=EARLIEST&limit=1'
    r = requests.get(url)
    data = r.json()
    for item in data.get('feed', []):
        row = {
            "Ticker": ticker,
            "Time Published": item.get('time_published'),
            "Overall Sentiment Score": item.get('overall_sentiment_score'),
            "Ticker Sentiment Score": item.get('ticker_sentiment')[0].get('ticker_sentiment_score'),
            #"Relevance Score": item.get('topics')[0].get('relevance_score'),
            #"Ticker Sentiment Label": item.get('ticker_sentiment')[0].get('ticker_sentiment_label')
        }
        print(row)  # Print each row as it's created
        return row

# Function to generate start and end times for the first day of each month
def generate_monthly_time_ranges(time_from, time_to):
    time_ranges = []
    current_time = datetime.strptime(time_to, '%Y%m%dT%H%M')
    while current_time >= datetime.strptime(time_from, '%Y%m%dT%H%M'):
        start_of_month = current_time.replace(day=1, hour=0, minute=0, second=0, microsecond=0)
        end_of_month = (start_of_month.replace(month=start_of_month.month % 12 + 1, year=start_of_month.year + (1 if start_of_month.month == 12 else 0))) - pd.Timedelta(days=1)
        time_ranges.append((start_of_month.strftime("%Y%m%dT%H%M"), end_of_month.strftime("%Y%m%dT%H%M")))
        current_time = start_of_month - pd.Timedelta(days=1)
    return time_ranges

# Generate start and end times for the first day of each month
time_ranges = generate_monthly_time_ranges(time_from, time_to)

# Fetch news sentiment data for each ticker and time range
output_file = "data/sentiment_monthly_data.csv"
for start_time, end_time in time_ranges:
    for ticker in tickers:
        row = fetch_news_sentiment(ticker, start_time, end_time)
        if row:
            df = pd.DataFrame([row])
            df.set_index('Time Published', inplace=True)
            df.to_csv(output_file, mode='a', header=not os.path.exists(output_file))  # Append if file exists, otherwise create it
            


{'Ticker': 'AAPL', 'Time Published': '20220301T080000', 'Overall Sentiment Score': -0.27746, 'Ticker Sentiment Score': '-0.24775'}
{'Ticker': 'NVDA', 'Time Published': '20220303T065619', 'Overall Sentiment Score': 0.010325, 'Ticker Sentiment Score': '0.007051'}
{'Ticker': 'AMZN', 'Time Published': '20220301T080000', 'Overall Sentiment Score': -0.337465, 'Ticker Sentiment Score': '-0.101763'}
{'Ticker': 'GOOG', 'Time Published': '20220301T080000', 'Overall Sentiment Score': -0.092611, 'Ticker Sentiment Score': '0.002617'}
{'Ticker': 'XOM', 'Time Published': '20220301T080000', 'Overall Sentiment Score': -0.291782, 'Ticker Sentiment Score': '-0.102637'}
{'Ticker': 'UNH', 'Time Published': '20220304T150007', 'Overall Sentiment Score': 0.277246, 'Ticker Sentiment Score': '0.129731'}
{'Ticker': 'PG', 'Time Published': '20220303T034400', 'Overall Sentiment Score': -0.113602, 'Ticker Sentiment Score': '-0.109784'}
{'Ticker': 'JNJ', 'Time Published': '20220304T112008', 'Overall Sentiment Score'